# Evaluate and Compare Optimizers

Some classes and functions to allow different optimization algorithms to be compared.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from dataclasses import dataclass
import skopt

import lpfgopt
lpfgopt.__version__

In [ ]:
from platform import python_version
python_version()

## Optimization Problem Definition

In [ ]:
@dataclass
class ConstrainedScalarOptimizationProblem():
    _name: str
    _guesses: list
    _input_dim: int
    _bounds: np.ndarray
    _global_minimum: np.ndarray

    def __init__(self, bounds, name=None, global_minimum=None):
        self._bounds = bounds
        self._input_dim = len(bounds)
        self._name = name
        self._global_minimum = global_minimum
        self.reset()

    def reset(self):
        self._guesses = []

    @property
    def name(self) -> str:
        return self._name

    @property
    def input_dim(self) -> tuple:
        return self._input_dim

    @property
    def bounds(self) -> np.ndarray:
        return self._bounds

    @property
    def global_minimum(self) -> np.ndarray:
        return self._global_minimum

    @property
    def nfev(self) -> int:
        return len(self._guesses)

    @property
    def guesses(self) -> list:
        return self._guesses

    @property
    def best_guess(self) -> tuple:
        return min(self._guesses)

    @staticmethod
    def cost_function_to_minimize(x, *args) -> float:
        # Implement cost function to minimize here
        cost = 0.0
        return cost

    def __call__(self, x, *args) -> float:
        assert np.all(
            (b[0] <= xi) and (xi <= b[1]) for b, xi in zip(self._bounds, x)
        )
        cost = self.cost_function_to_minimize(x, *args)
        self._guesses.append((cost, x))
        return cost


### Example 1. Toy 1D Problem

In [ ]:
class Toy1DProblem(ConstrainedScalarOptimizationProblem):

    def __init__(self):
        bounds = [(-5.0, 5.0)]
        name = "Toy1DProblem"
        super().__init__(bounds, name=name, global_minimum=[2.5085382557867626])

    @staticmethod
    def cost_function_to_minimize(x) -> float:
        return 1.0 / (-0.05 * x[0] ** 2 - np.cos(x[0]) + 0.25 * np.sin(3 * x[0] + 0.8) + 5)


# Test problem instance
problem = Toy1DProblem()
assert str(problem) == (
    "Toy1DProblem(_name='Toy1DProblem', _guesses=[], _input_dim=1, "
    "_bounds=[(-5.0, 5.0)], _global_minimum=[2.5085382557867626])"
)
assert problem.bounds == [(-5.,  5.)]
assert problem.input_dim == 1
assert problem.nfev == 0
assert problem.guesses == []
assert problem([0.5]) == 0.23275605031813504
assert problem.guesses == [(0.23275605031813504, [0.5])]
assert problem.nfev == 1
assert problem([-5]) == 0.3108649328945798
assert problem([5]) == 0.29041392127738885
assert problem.nfev == 3
assert problem.best_guess == (0.23275605031813504, [0.5])

# Find global minimum using Scipy and a good initial guess
sol = scipy.optimize.minimize(problem, x0=2.5, bounds=problem.bounds, tol=1e-15)
assert sol.status == 0
print(sol.fun, sol.x.item())
assert np.array_equal(problem.global_minimum, sol.x)

In [ ]:
X = np.linspace(-5, 5, 100).reshape(1, -1)
Y = problem(X)

min_pt = np.array([problem.global_minimum[0], problem(problem.global_minimum)])

plt.plot(X.T, Y.T)
plt.plot(*min_pt, 'ko')
plt.annotate(f'Min: {min_pt.round(3)}', min_pt, xytext=(10, 0),
             textcoords='offset points', va='center')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Function to minimize')
plt.grid()

In [ ]:
def solve_problem_with_optimizer(problem, minimizer, *args, **kwargs):
    problem.reset()
    sol = minimizer(problem, *args, **kwargs)
    return sol

rng = np.random.default_rng(0)
x0 = rng.uniform(*zip(*problem.bounds))
sol = solve_problem_with_optimizer(problem, scipy.optimize.minimize, x0, bounds=problem.bounds)
sol

In [ ]:
def convergence_plot(problem, title=None):
    if title is None:
        title = f'Optimizer Convergence - {problem.name}'
    fevals = np.array([item[0] for item in problem.guesses])
    fig = plt.figure(figsize=(7, 2.5))
    ax = fig.gca()
    plt.semilogy(fevals, marker='.')
    plt.xlabel('Number of function evaluations')
    plt.ylabel('f(x)')
    plt.grid()
    plt.title(title)
    return ax

convergence_plot(problem)
plt.tight_layout()
plt.show()

In [ ]:
solve_problem_with_optimizer(problem, lpfgopt.minimize, problem.bounds)

In [ ]:
convergence_plot(problem)
plt.tight_layout()
plt.show()

In [ ]:
# TODO: Plot variance of convergence results

from collections import Counter

def solve_problem_with_optimizer_n_repeats(
    problem, minimizer, n_repeats, *args, decimals=6, **kwargs
):

    solutions = []
    fun_evals = []
    for i in range(n_repeats):
        problem.reset()
        sol = minimizer(problem, *args, **kwargs)
        solutions.append(tuple(round(float(xi), decimals) for xi in sol.x))
        fun_evals.append(np.array([f[0] for f in problem.guesses]))
    unique_solutions = Counter(solutions)
    return fun_evals, unique_solutions

fun_evals, unique_solutions = solve_problem_with_optimizer_n_repeats(problem, lpfgopt.minimize, 20, problem.bounds, tol=1e-6)
unique_solutions

In [ ]:
def convergence_plot_n_repeats(fun_evals, title=None, marker='auto', color='tab:blue', alpha=0.25):

    n_repeats = len(fun_evals)

    if title is None:
        title = f'Optimizer Convergence - {n_repeats} Iterations'

    # Find the maximum length among all sequences
    max_len = max(len(f) for f in fun_evals)

    # Extend all sequences to the same length by repeating their final value
    series_array = np.full((n_repeats, max_len), np.nan)
    for i, f in enumerate(fun_evals):
        n = len(f)
        series_array[i, :n] = f
        series_array[i, n:] = f[-1]

    # Calculate min, max, and median across all series at each iteration
    min_vals = np.min(series_array, axis=0)
    max_vals = np.max(series_array, axis=0)
    median_vals = np.median(series_array, axis=0)

    # Create the plot
    fig = plt.figure(figsize=(7, 2.5))
    ax = fig.gca()

    # Plot the median line
    if marker == 'auto':
        marker = '.' if max_len < 20 else None
    x = np.arange(max_len)
    ax.semilogy(x, median_vals, marker=marker, color=color, label='Median')

    # Fill between min and max
    ax.fill_between(x, min_vals, max_vals, alpha=alpha, color=color, label='Min-Max Range')

    plt.xlabel('Number of function evaluations')
    plt.ylabel('f(x)')
    plt.grid()
    plt.title(title)
    plt.legend()
    
    return ax

In [ ]:
ax = convergence_plot_n_repeats(fun_evals)

## Bayesian Optimization Minimizer

In [ ]:
# Run Bayesian optimization
problem.reset()
res = skopt.gp_minimize(
    problem,            # the function to minimize
    problem.bounds,     # the bounds on each dimension of x
    n_calls=50,
    noise=1e-10,
    random_state=0
)
res

In [ ]:
res['x'], res['fun']

In [ ]:
problem.best_guess

In [ ]:
fun_evals, unique_solutions = solve_problem_with_optimizer_n_repeats(
    problem, skopt.gp_minimize, 20, problem.bounds, noise=1e-10, n_calls=50
)
unique_solutions

In [ ]:
ax = convergence_plot_n_repeats(fun_evals)